In [ ]:
# Import necessary libraries
import requests
import pandas as pd
import numpy as np
import json

# API URL: The URL of the Data API endpoint.
# If you don't have this information, you can get it on DPH -> Data Product -> Access Options -> Data APIs -> API Endpoint
api_url = "https://dataos-training.dataos.app/lens2/api/public:productaffinity/v2/load"

# API Key: Your DataOS API key. 
# If you don't have, you can get it on DPH -> Data Product -> Access Options -> Data APIs -> API Endpoint -> Click here to generate API Access Token
apikey = ''   


# API payload, enter YOUR_QUERY here.
# Query Example: This is how the structure of your query should look like. 
payload = json.dumps({
    "query": {
      "measures": [
        "product.total_products",
        "purchase.purchase_frequency",
        "purchase.total_spend",
        "purchase.cross_sell_opportunity_score",
        "customer.total_customers" 
      ],
      "dimensions": [
        "customer.customer_id",
        "product.product_category",
        "purchase.purchase_date"
      ],
      "segments": [],
      "filters": [],
      "timeDimensions": [],
      "limit": 10,
      "offset": 0
    }
})



# Headers
headers = {
    'Content-Type': 'application/json',
    'apikey': apikey
}

# Fetch data from API
def fetch_data_from_api(api_url, payload, headers=None):
    response = requests.post(api_url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['data'])  # Create DataFrame
        return df
    else:
        print(f"Error: {response.status_code}")
        return None

# Create Product Affinity Matrix
def calculate_product_affinity(df):
    # Step 1: Create a customer-product matrix
    customer_product_matrix = pd.crosstab(df['customer.customer_id'], df['product.product_category'])

    # Step 2: Create a co-occurrence matrix (affinity matrix)
    product_affinity_matrix = np.dot(customer_product_matrix.T, customer_product_matrix)

    # Convert to DataFrame for easier handling
    affinity_df = pd.DataFrame(product_affinity_matrix, 
                               index=customer_product_matrix.columns, 
                               columns=customer_product_matrix.columns)

    # Remove self-affinity (i.e., diagonal entries)
    np.fill_diagonal(affinity_df.values, 0)

    return affinity_df

# Main execution
if __name__ == "__main__":
    data = fetch_data_from_api(api_url, payload, headers=headers)

    if data is not None:
        print("Data Frame Created:")
        print(data.head())  # Show the first few rows of the DataFrame

        # Calculate product affinity
        product_affinity_df = calculate_product_affinity(data)

        print("Product Affinity Matrix:")
        print(product_affinity_df)
    else:
        print("Failed to fetch data.")